In [ ]:
"""
remember, if your file path has backslashes where spaces are because you pasted it from terminal, remove those!!

For the psychopy csv:
"Stimulus" is the name of the csv column for event type
"Marker Timestamp" is the name of the csv column for when the event occured
The names should be changed in the code or in the csv if different names are being used

For the following lines of code:
eeg_df = pd.read_csv("/Users/username/Desktop/folder/eegfilename.csv")  #put the file path to your eeg csv here
beh_raw = pd.read_csv("/Users/username/Desktop/folder/psychopyfilename.csv") #put the filepath to your psychopy csv here
the filepaths written inside the "" should be replaced with the actual filepaths to your csv data

"""

In [ ]:
pip install mne seaborn bokeh


In [ ]:
import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import output_notebook
sns.set_style("darkgrid")
output_notebook()


In [22]:
class ERPRecording:

    def __init__(self, eeg_df, behav_df, tmin=-0.2, tmax=1.0, sfreq=256):
        self.sfreq = sfreq
        self.tmin = tmin
        self.tmax = tmax
        self.erp_dict = {}
        self.grand_avg_dict = {}

        stim_labels = behav_df["Stimulus"].unique()
        self.event_dict = {label: i + 1 for i, label in enumerate(stim_labels)}

        # Build marker channel
        marker_series = pd.Series(0, index=eeg_df.index)
        for i, row in behav_df.iterrows():
            ts = row["marker_ts"]
            idx = (np.abs(eeg_df["timestamps"] - ts)).idxmin()
            marker_series.iloc[idx] = self.event_dict[row["Stimulus"]] 

        # Build MNE Raw
        channels = eeg_df.drop(columns=["timestamps"]).columns.tolist()
        eeg_data = eeg_df.drop(columns="timestamps").T.values
        eeg_data = np.vstack([eeg_data, marker_series.values[np.newaxis, :]])

        info = mne.create_info(ch_names=channels + ["markers"], sfreq=sfreq,
                               ch_types=["eeg"] * len(channels) + ["stim"])
        raw = mne.io.RawArray(eeg_data, info)
        raw.filter(1, 30)

        # Epochs
        events = mne.find_events(raw)
        epochs = mne.Epochs(raw, events, event_id=self.event_dict, tmin=tmin, tmax=tmax, preload=True)

        # Store ERPs
        for label in stim_labels:
            evoked = epochs[label].average()
            self.erp_dict[label] = [evoked]
            self.grand_avg_dict[label] = evoked

    def plot(self, electrode="AF8", markers=None):
        if markers is None:
            markers = list(self.erp_dict.keys())

        for label in markers:
            df = self.grand_avg_dict[label].to_data_frame()
            if electrode not in df.columns:
                print(f"⚠️ Electrode '{electrode}' not found in ERP data.")
                continue
            plt.plot(df["time"], df[electrode], label=label)

        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude (μV)")
        plt.title(f"ERP at {electrode}")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()


In [ ]:
eeg_df = pd.read_csv("/Users/username/Desktop/folder/eegfilename.csv")  #put the file path to your eeg csv here
beh_raw = pd.read_csv("/Users/username/Desktop/folder/psychopyfilename.csv") #put the filepath to your psychopy csv here

# Convert Unix timestamps (in seconds) to datetime format
eeg_df['timestamps'] = pd.to_datetime(eeg_df['timestamps'], unit='s')
beh_raw['marker_ts'] = pd.to_datetime(beh_raw['Marker Timestamp'], unit='s')


In [ ]:
session = ERPRecording(eeg_df, beh_raw, tmin=-0.3, tmax=1.0)


In [ ]:
#plot of the AF7 electrode according to Stimulus type
conditions = beh_raw["Stimulus"].unique().tolist()
session.plot(electrode="AF7", markers=conditions)

In [ ]:
#plot of the AF8 electrode according to Stimulus type
conditions = beh_raw["Stimulus"].unique().tolist()
session.plot(electrode="AF8", markers=conditions)

In [ ]:
#plot of the TP9 electrode according to Stimulus type
conditions = beh_raw["Stimulus"].unique().tolist()
session.plot(electrode="TP9", markers=conditions)

In [ ]:
#plot of the TP10 electrode according to Stimulus type
conditions = beh_raw["Stimulus"].unique().tolist()
session.plot(electrode="TP10", markers=conditions)